### <u> Generate a sonification with an audio caption </u>
**Import the relevant modules:**

In [ ]:
%matplotlib inline
%reload_ext autoreload 
%autoreload 2
from strauss.sonification import Sonification
from strauss.sources import Events
from strauss import channels
from strauss.score import Score
from strauss.tts_caption import render_caption
import numpy as np
from strauss.generator import Sampler
import os

**Generate the sonification (a short sequence of glockenspiel notes):**

In [ ]:
# platform agnostic absolute path for samples...
import strauss
sample_path = '/'.join(strauss.__file__.split('/')[:-3] +['data','samples','glockenspiels'])

# setup used in stars appearing example
chords = [['Db3','Gb3', 'Ab3', 'Eb4','F4']]
length = 6
system = 'mono'
score =  Score(chords, length)

maplims =  {'time': ('0', '150'),
            'pitch' : ('0', '100'),
           'phi':('0','100'),
            'theta':('0','100')}

events = Events(maplims.keys())

data = {'pitch':np.arange(5),
        'time':np.arange(5),
       'phi': np.arange(5),
       'theta': np.arange(5)}


generator = Sampler(sample_path)

events.fromdict(data)
events.apply_mapping_functions(map_lims=maplims)

**The caption is expected in the form of a text file. Here, we write a temporary text file for the audio caption, then read it in and delete it afterwards.**

In [ ]:
# Read a caption in from a text file
with open('temp_text.txt', 'w') as f:
    f.write(' In the following sonification, a glockenspiel is used to represent stars of varying colour.')
with open('temp_text.txt', 'r') as f:
    caption = f.read().replace('\n', '')
os.remove('temp_text.txt')

**Generate text-to-speech (TTS) for the caption.**

In [ ]:
# render at default 48 kHz rate
soni = Sonification(score, events, generator, system,
                    caption=caption, samprate=48000,
                    model='tts_models/en/jenny/jenny')
soni.render()
soni.notebook_display()

### <u> About TTS </u>
**Other TTS models are available in several languages. We can list them and try some out.**

In [ ]:
import sys
print(sys.version_info)
from TTS.api import TTS
from IPython.display import Audio

# List models
TTS.list_models()

**It can take a while to download a model for the first time.**

In [ ]:
# Test example 1: the following is a male German voice.
OUTPUT_PATH = 'tts_german.wav'
tts = TTS(model_name='tts_models/de/thorsten/vits', progress_bar=False, gpu=False)
tts.tts_to_file(text=' Der flinke braune Fuchs sprang über die faulen Hunde.', file_path=OUTPUT_PATH)
Audio(OUTPUT_PATH, autoplay=False)

In [ ]:
# Test example 2: the following is a female US-English voice.
OUTPUT_PATH = 'tts_english.wav'
tts = TTS(model_name='tts_models/en/ljspeech/tacotron2-DDC', progress_bar=False, gpu=False)
tts.tts_to_file(text=' The quick brown fox jumped over the lazy dogs.', file_path=OUTPUT_PATH)
Audio(OUTPUT_PATH, autoplay=False)

**TTS ignores anything it doesn't recognise, such as Greek letters and some mathematical symbols. It can also struggle with multi-digit numbers. It's best to write these out long-hand. Here is an example:**

In [ ]:
tts = TTS(model_name='tts_models/en/jenny/jenny', progress_bar=False, gpu=False)
OUTPUT_PATH = 'tts_lya.wav'
tts.tts_to_file(text=" The Lyman-α resonance is 1216 Å. The Lyman alpha resonance is twelve hundred and sixteen angstroms.",
                file_path=OUTPUT_PATH)
Audio(OUTPUT_PATH, autoplay=False)

## <u> Sandbox </u>